# AI CA3
## Naive Bayes Classifiers
## Amirhossein Alizad 810197546
<strong> we are going to analyze the comments on a given shop as a train file and then predict for each comment in a given test file whether the user recommended it or not</strong>

<strong> by reading the comment train file, we calculate the probability of each word and divide them into two categories, recommended and not_recommended. then by doing the same for comments in test file and using bayes rule to find out whether it is recommended or not.</strong>

In [1]:
#imports
from hazm import *
import pandas as pd
import time

<strong> using hazm library to normalize comments and pandas to read and operate on csv files</strong>

# Phase 1

In [2]:
#globals
global sym
sym = ['.', '!', '،', '?']

global stopwrd
stopwrd = stopwords_list()

global RECOMMENDED
RECOMMENDED = 0

global NOT_RECOMMENDED
NOT_RECOMMENDED = 1


<strong> 'sym' is an array consisting of symbols that does not concern the recommendation of a product according to a comment and so we will ignore it.<br>
just likethe sym, we have a list of stop words which are the words that happen alot and usually does not change the meaning of the sentence so we will ignore them too.</strong>

In [3]:
def normalize_comment(sentence):
    normalizer = Normalizer()
    sentence = normalizer.normalize(sentence)
    lst1 =  word_tokenize(sentence)
    stemmer = Stemmer()
    lst2 = [stemmer.stem(i) for i in lst1]
    return lst2

## 1)
<strong>to normalize a comment we can do 3 operations on a sentence or a word:<br>
1)normalize: this function is used for replacing space with half-space. this function makes data proccesing easier<br><br>
2)stemming: it creates the base of a word, may be wrong some times, but still helps to normalize the sentence. for example words: زیبایی and زیبا are two different words but when stemmed, they both turn into زیبا meaning that the sentence they were both in may have the same meaning.<br><br>
3)lemmatization: just like stemming but works on verbs and returns the base form of a verb. for example میخواهم and میخواستم refer to a different time and technically are different but by lemmatizing they both turn into خواه#خواست causing them to have the same effect on a sentence.<br><br>
stemming usually removes the extra end of a word like plural signs, etc.lemmatization also does the same and also returns the root and base of the word or a verb.one of the errors of stem is for exaple the words ending with م. the stemmer changes the word بادام into بادا causing it to be completely meaningless. and also lemmatizer takes much more time than stemming (this is concluded from runnig and testing with both functions and there were times that lemmatizer did not even finish the operation)

# Phase 2

In [4]:
def eval_dictionary_without_smoothing(dic, lst, recommended):
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            if i in dic[recommended]:
                dic[recommended][i] += 1
            else:
                dic[recommended][i] = 1
                
def eval_dictionary_with_smoothing(dictionary, lst):
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            if i not in dictionary[RECOMMENDED]:
                dictionary[RECOMMENDED][i] = 1
            else:
                dictionary[RECOMMENDED][i] += 1
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            if i not in dictionary[NOT_RECOMMENDED]:
                dictionary[NOT_RECOMMENDED][i] = 1
            else:
                dictionary[NOT_RECOMMENDED][i] += 1

<strong> given the normalized sentence, we evaluate our dictionary (wether we use smoothing or not) by the words in the sentence. if the word is not in stop words list and not a symbol then we can use it in our dictionary</strong>

In [5]:
def calc_p_without_smoothing(p, lst, rec_p, not_rec_p):
    rec = p[RECOMMENDED]
    not_rec = p[NOT_RECOMMENDED]
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            if i in rec_p:
                rec *= rec_p[i]
            else:
                rec = 0
                break
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            if i in not_rec_p:
                not_rec *= not_rec_p[i]
            else:
                not_rec = 0
                break
    return rec, not_rec

def calc_p_with_smoothing(lst, p, dictionary):
    rec = p[RECOMMENDED]
    not_rec = p[NOT_RECOMMENDED]
    rec_p = dict()
    not_rec_p = dict()
    rec_count = [0]
    not_rec_count = [0]
    for i in dictionary[RECOMMENDED]:
        rec_count[0] += dictionary[RECOMMENDED][i]
    for i in dictionary[NOT_RECOMMENDED]:
        not_rec_count[0] += dictionary[NOT_RECOMMENDED][i]
    for i in dictionary[RECOMMENDED]:
        rec_p[i] = dictionary[RECOMMENDED][i] / rec_count[0]
    for i in dictionary[NOT_RECOMMENDED]:
        not_rec_p[i] = dictionary[NOT_RECOMMENDED][i] / not_rec_count[0]
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            rec *= rec_p[i]
    for i in lst:
        if not(i in stopwrd) and not(i in sym):
            not_rec *= not_rec_p[i]
    return rec, not_rec

<strong> we calculate the probability of every word, if we're using smoothing,we find all the words that are not in the recommended and not recommended dictionary and increase them by 1 every time we see them. then the probability for every word is : $count_i / \sum count_j$.<br>
after calculating the probability of every word from train data, we can find wether a comment in test data is recommended or not.
in smoothing we first find the words that are not in our dictionary and intialize them to 1, and every time we see it again, we increase it.

# 2)
<strong>
1) ‫‪evidence: P(comment) is the evidence which is not useful in this assignment.<br>
2) likelihood: P(comment|recommended) is the likelihood. we calculate it by multiplying all P(word|recommended).<br>
3) prior: P(recommended or not recommended) is the prior probability of class. we calculate it by counting all of the recommended or not recommended comments in test data and divide it by number of all comments or p(recommended) = 1 - p(not recommended)<br>
4) posterior: P(recommended|comment) is the posterior probability. it is the probability of being recommended or not recommended given the comment. we calculate it by multiplying likelihood and prior.<br>
</strong>

# 3)
<strong>in naive bayes we have a bag of words which are fixed and do not change, when we find a word in a comment that is not for example in th recommended list, we say for sure that this comment is not recommended because <br>$p[recommended] = p[word] * p[recommended]$ and because we said that the word is nott in our list then $p[word] = 0$ thus $p[recommended] = 0$ so it is not_recommended.

# 4)
<strong> additive smoothing is a technique to smooth data.it is used to solve the problem of zero probability. to implement it we add 1 to the count of all words in the train data, when a word is not in the train list, its probability will not be zero as its count is now 1 and when we see it again we will increase it so it is not zero any more and the problem is solved.it helps the problem in a way that when for example we have a comment: لذت بخش بود
<br> but we dont have the word لذت in our list so the probability of recommended will be 0 and it will be resulted as not recommended. as you can understand this comment in a way says that it recommends the product but we predicted it as not recommended because of 0 probability. by using smoothing this problem is solved and the comment will be (with a higher probability) resulted as recommended.

In [6]:
def train_with_pre(row, dic, rec_count, not_rec_count):
    lst = normalize_comment(row['comment'])
    comment_class = row['recommend']
    if comment_class == "recommended":
        recommended = RECOMMENDED
        rec_count[0] += 1
    else:
        recommended = NOT_RECOMMENDED
        not_rec_count[0] += 1
    eval_dictionary_without_smoothing(dic, lst, recommended)
    
def train_without_pre(row, dic, rec_count, not_rec_count):
    sentence = row['comment']
    comment_class = row['recommend']
    if comment_class == "recommended":
        recommended = RECOMMENDED
        rec_count[0] += 1
    else:
        recommended = NOT_RECOMMENDED
        not_rec_count[0] += 1
    lst =  word_tokenize(sentence)
    eval_dictionary_without_smoothing(dic, lst, recommended)

<strong> these are the functions that evaluate dictionaries and process on the comments which as you can see the first on has a pre process function(normalize_comment()) and the second one does not</strong>


In [7]:
def test_with_pre_without_smooth(row, p, rec_p, not_rec_p, all_of, correct):
    lst = normalize_comment(row['comment'])
    comment_class = row['recommend']
    rec, not_rec = calc_p_without_smoothing(p, lst, rec_p, not_rec_p)
    if(rec >= not_rec):
        all_of[0] += 1
    if (rec >= not_rec and comment_class == "recommended") or (rec < not_rec and comment_class == "not_recommended"):
        if(rec >= not_rec):
            correct[0] += 1
        return True
    else:
        return False

def test_with_pre_with_smooth(row, p, dictionary, all_of, correct, wrong):
    comment_class = row['recommend']
    lst = normalize_comment(row['comment'])
    rec = p[RECOMMENDED]
    not_rec = p[NOT_RECOMMENDED]
    eval_dictionary_with_smoothing(dictionary, lst)
    rec, not_rec = calc_p_with_smoothing(lst, p, dictionary)
    if(rec >= not_rec):
        all_of[0] += 1
    if (rec >= not_rec and comment_class == "recommended") or (rec < not_rec and comment_class == "not_recommended"):
        if(rec >= not_rec):
            correct[0] += 1
        return True
    else:
        wrong.append((row['title'], row['comment'], row['recommend']))
        return False

def test_without_pre_with_smooth(row, p, dictionary, all_of, correct):
    sentence = row['comment']
    comment_class = row['recommend']
    lst =  word_tokenize(sentence)
    rec = p[RECOMMENDED]
    not_rec = p[NOT_RECOMMENDED]
    eval_dictionary_with_smoothing(dictionary, lst)
    rec, not_rec = calc_p_with_smoothing(lst, p, dictionary)
    if(rec >= not_rec):
        all_of[0] += 1
    if (rec >= not_rec and comment_class == "recommended") or (rec < not_rec and comment_class == "not_recommended"):
        if(rec >= not_rec):
            correct[0] += 1
        return True
    else:
        return False
def test_without_pre_without_smooth(row, p, rec_p, not_rec_p, all_of, correct):
    sentence = row['comment']
    comment_class = row['recommend']
    lst =  word_tokenize(sentence)
    rec, not_rec = calc_p_without_smoothing(p, lst, rec_p, not_rec_p)
    if(rec >= not_rec):
        all_of[0] += 1
    if (rec >= not_rec and comment_class == "recommended") or (rec < not_rec and comment_class == "not_recommended"):
        if(rec >= not_rec):
            correct[0] += 1
        return True
    else:
        return False   

<strong>as you can see above, there is a function for every kind of prediction. we use these functions with apply function in pandas and apply it on every row in a Dataframeand in these functions if we predicted that it was recommended we increas variable 'all_of' and if it was correct we increase variable 'correct'</strong>

# Phase 3
# 5)
<strong>precision: to calculate the accuracy of our prediction, we divide all the right recommended types we predicted by all the recommended types (including the wrong ones). suppose we predicted only one of the comments and we predicted it correct then we get 100% accuracy but if we focus we'll know that we're wrong.<br><br>
recall: we divide all the right recommended types we predicted by all the recommended types in the correct answer(which is given in the test file).if we predict all the types to be recommended then we will get 100% recall which is also not correct.<br>
</strong>
# 6)
<strong>
F1: we use harmonic mean for F1.The harmonic mean helps to find multiplicative or divisor relationships between fractions.

In [8]:
def print_results(df, df_test, correct, all_of):
    accuracy = formatted_float = "{:.2f}".format(100 * df.value_counts()[True] / df.count())
    precision = formatted_float = "{:.2f}".format(100 * correct[0] / all_of[0])
    recall = formatted_float = "{:.2f}".format(100 * correct[0] / df_test['recommend'].value_counts()['recommended'])
    F1 = formatted_float = "{:.2f}".format(2 * (float(precision) * float(recall)) / (float(precision) + float(recall)))
    print(" accuracy : %", accuracy)
    print("precision : %", precision)
    print("   recall : %", recall)
    print("       F1 : %", F1)

In [9]:
def operation(op, wrong = None):
    t1 = time.time()
    print("-----------------------------------------------------")
    df_train = pd.read_csv("comment_train.csv")
    rec_count = [0]
    not_rec_count = [0]
    all_of = [0]
    correct = [0]
    dictionary = [dict() for i in range(2)]
    
    if op == 'pre_smooth':
        df_train.apply(lambda x : train_with_pre(x, dictionary, rec_count, not_rec_count), axis = 1)
        count = df_train.count()[0]
        p = [rec_count[0], not_rec_count[0]] / count
        df_test = pd.read_csv("comment_test.csv")
        df = df_test.apply(lambda x : test_with_pre_with_smooth(x, p, dictionary, all_of, correct, wrong), axis = 1)
        print(op, ":")
        print_results(df, df_test, correct, all_of)
        
    elif op == 'smooth':
        df_train.apply(lambda x : train_without_pre(x, dictionary, rec_count, not_rec_count), axis = 1)
        count = df_train.count()[0]
        p = [rec_count[0], not_rec_count[0]] / count
        df_test = pd.read_csv("comment_test.csv")
        df = df_test.apply(lambda x : test_without_pre_with_smooth(x, p, dictionary, all_of, correct), axis = 1)
        print(op, ":")
        print_results(df, df_test, correct, all_of)
        
    elif op == 'pre':
        df_train.apply(lambda x : train_with_pre(x, dictionary, rec_count, not_rec_count), axis = 1)
        count = df_train.count()[0]
        p = [rec_count[0], not_rec_count[0]] / count
        rec_p = dict()
        not_rec_p = dict()
        rec_count = [0]
        not_rec_count = [0]
        for i in dictionary[RECOMMENDED]:
            rec_count[0] += dictionary[RECOMMENDED][i]
        for i in dictionary[NOT_RECOMMENDED]:
            not_rec_count[0] += dictionary[NOT_RECOMMENDED][i]
        for i in dictionary[RECOMMENDED]:
            rec_p[i] = dictionary[RECOMMENDED][i] / rec_count[0]
        for i in dictionary[NOT_RECOMMENDED]:
            not_rec_p[i] = dictionary[NOT_RECOMMENDED][i] / not_rec_count[0]
        df_test = pd.read_csv("comment_test.csv")
        df = df_test.apply(lambda x : test_with_pre_without_smooth(x, p, rec_p, not_rec_p, all_of, correct), axis = 1)
        print(op, ":")
        print_results(df, df_test, correct, all_of)
        
    else:
        df_train.apply(lambda x : train_without_pre(x, dictionary, rec_count, not_rec_count), axis = 1)
        count = df_train.count()[0]
        p = [rec_count[0], not_rec_count[0]] / count
        rec_p = dict()
        not_rec_p = dict()
        rec_count = [0]
        not_rec_count = [0]
        for i in dictionary[RECOMMENDED]:
            rec_count[0] += dictionary[RECOMMENDED][i]
        for i in dictionary[NOT_RECOMMENDED]:
            not_rec_count[0] += dictionary[NOT_RECOMMENDED][i]
        for i in dictionary[RECOMMENDED]:
            rec_p[i] = dictionary[RECOMMENDED][i] / rec_count[0]
        for i in dictionary[NOT_RECOMMENDED]:
            not_rec_p[i] = dictionary[NOT_RECOMMENDED][i] / not_rec_count[0]
        df_test = pd.read_csv("comment_test.csv")
        df = df_test.apply(lambda x : test_without_pre_without_smooth(x, p, rec_p, not_rec_p, all_of, correct), axis = 1)
        print(op, ":")
        print_results(df, df_test, correct, all_of)
    print("     time : ", time.time() - t1, "seconds")

# 7)
<strong> from the answers shown below we find out that by the normalization that we do in our preprocess and additive smoothing, smoothing alone is almost better than doing both preprocess and additive smoothing and they both are far better than only doing preprocess or neither.
<br>NOTE* : this conclusion is relative to the implementation of preprocess and additive smoothing and by doing some more preprocess like lemmatization, doing both of the operations may be better than doing only one of them or neither.</strong>

# 7-a)

In [10]:
wrong = []
operation('pre_smooth', wrong)

-----------------------------------------------------
pre_smooth :
 accuracy : % 90.50
precision : % 90.30
   recall : % 90.75
       F1 : % 90.52
     time :  6.5173656940460205 seconds


# 7-b)

In [11]:
operation('smooth')

-----------------------------------------------------
smooth :
 accuracy : % 91.25
precision : % 91.25
   recall : % 91.25
       F1 : % 91.25
     time :  6.554788112640381 seconds


# 7-c)

In [12]:
operation('pre')

-----------------------------------------------------
pre :
 accuracy : % 86.12
precision : % 80.29
   recall : % 95.75
       F1 : % 87.34
     time :  2.504483222961426 seconds


# 7-d)

In [13]:
operation('none')

-----------------------------------------------------
none :
 accuracy : % 85.50
precision : % 79.34
   recall : % 96.00
       F1 : % 86.88
     time :  1.4043176174163818 seconds


In [14]:
wrong_df = pd.DataFrame(wrong, columns =['title', 'comment', 'recommend']) 
wrong_df.head(5)

,title,comment,recommend
0,وری گود,تازه خریدم یه مدت کار بکنه مشخص میشه کیفیت قطعاتش,recommended
1,زیاد مناسب نیست رنگ پس میده یه وقتایی موقع نوشتن,با این قیمت گزینه های بهتری هم میشه گرفت.\r\nر...,not_recommended
2,نقد پس از خرید,سلام ، راحت شدم از کابل شارژ ، توصیه میشود به ...,recommended
3,خیالم راحت شد,فندک قبلیم مدام فیوز میسوزوند و یک بار شارژر م...,recommended
4,خرید 2 عدد و مغایرت سریaو bو تفاوت دو مدل,، دوستان عزیز دقت کنند در مورد خرید این پاوربا...,not_recommended


In [15]:
wrong[1][1]

'با این قیمت گزینه های بهتری هم میشه گرفت.\r\nروان مینویسه ولی زیاد مناسب نیست و رنگ پس میده یه وقتایی موقع نوشتن'

as you can see from above this comment was supposed to be not_recommended but we predicted that it would be recommended. the reason is that in the sentence : <br>
با این قیمت گزینه های بهتری هم میشه گرفت.\r\nروان مینویسه ولی زیاد مناسب نیست و رنگ پس میده یه وقتایی موقع نوشتن
<br>
it has a negative meaning but the words in the comment mostly are not negative like: مناسب will be taken as a positive adj or بهتری will be changed to بهتر which is positive, these things will cause a comment to be predicted wrongly.